## <font color=orange>Extracting Subduction Zone Kinematics & Parameters (pyGPlates method)</font>

#### <font color=blue>Notebook 1</font>

**This notebook can be used to extract subduction zone kinematics and parameters from a chosen plate model. These include:**
- overriding and subducting plate ID's
- arc length
- lat and lon
- time
- convergence rate and obliquity
- migration rate and obliquity 
- age of the subducting seafloor
- distance to nearest continent

**It illustrates usage of the updated subduction zone kinematics pygplates method, using the Müller et al AREPS model:**

- implemented to work directly on the plate model gpml files (whereas before, you had to export the resolved topologies at 1Myr increments first)
- this notebook also interpolates the age of the subducting plate age from age grids if provided
- within the notebook data are stored in pandas dataframes, and there is an illustration of how to export them to both csv or hdf5 files

This was the first in a series of notebooks used to generate the data presened in the following manuscript:

*'Subduction history reveals Cretaceous slab superflux as a possible cause for the mid-Cretaceous plume pulse and superswell events'* by Madison East, R Dietmar Müller, Simon Williams, Sabin Zahirovic and Christian Heine


(Notebook written by Simon Williams with minor modifications made by Madison East)

**<font color=red>The age grids are required to run this notebook.</font>**

The grids can be found at https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.11.zip. Download and unzip the zip file into the "data" folder before running this notebook.

#### Reference:
East, M., Müller, R. D., Williams, S., Zahirovic, S. and Heine, C., Subduction history reveals Cretaceous slab superflux as a possible cause for the mid-Cretaceous plume pulse and superswell events, Gondwana Research, in review.


In [9]:
# This cell loads in the required files for the chosen plate model and defines some required functions #
#########################################################################################################

import pygplates
import sys
sys.path.append('../libs/')
import subduction_convergence as sc #(required code stored in a .py file)
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import gridspec
from scipy.io import netcdf_file as netcdf
import scipy.interpolate as spi
from mpl_toolkits.basemap import Basemap
import inpaint
import pandas as pd

%matplotlib inline

###################################################################
# Define Input files for Muller 2016 AREPS model (or chosen model)
###################################################################
# Specify the name of the GPlates rotation file
rotation_filename = 'Data/Global_EarthByte_230-0Ma_GK07_AREPS.rot'

# input topologies to be used with the subduction_convergence script
input_topology_filename = ['Data/Global_EarthByte_230-0Ma_GK07_AREPS_PlateBoundaries.gpmlz',\
             'Data/Global_EarthByte_230-0Ma_GK07_AREPS_Topology_BuildingBlocks.gpmlz']

# The list array here specifies the age grid files, 
# assuming that the age grids are all together in a folder, with common filenames that
# differ only in the integer number somewhere in the filename that corresponds to the age in Ma
# The two parts of the list specify the parts of the filename before and after the number
# For example:
# ['/path_to_age_grids/age_grids_at_time_','_Ma.grd]
#agegrid_filename = ['../AgeGrids_most_recent/agegrid_final_nomask_','.grd']
agegrid_filename = \
    ['Data/Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_etal_2016_AgeGrid-',
     '.nc']

# Static polygons to determine whether subduction segment is adjacent to continent or not
static_polygon_filename = 'Data/StaticPolygons/Global_EarthByte_GPlates_PresentDay_StaticPlatePolygons_2015_v1.shp'
static_polygon_features = pygplates.FeatureCollection(static_polygon_filename)
continental_polygon_features = []
for feature in static_polygon_features:
    if feature.get_feature_type() == pygplates.FeatureType.gpml_closed_continental_boundary:
        continental_polygon_features.append(feature)


######################################################
rotation_model = pygplates.RotationModel(rotation_filename)

# specify time range and resolution for plots
threshold_sampling_distance_radians = np.radians(0.5)

# Define the time snapshots at which to get the subduction zone properties
min_time = 0.
max_time = 230.
time_step = 1.

# Set the delta time for velocity calculations
velocity_delta_time = 1.

# Typically the achor plate id should be 0
anchor_plate_id = 0


#################################################
# Included functions
#################################################
def sample_grid_using_scipy(x,y,grdfile):
    
    data=netcdf(grdfile,'r')
    try:
        lon = np.copy(data.variables['x'][:])
        lat = np.copy(data.variables['y'][:])
    except:
        lon = np.copy(data.variables['lon'][:])
        lat = np.copy(data.variables['lat'][:])
    
    Zg = data.variables['z'][:]
    
    test = inpaint.fill_ndimage(Zg)
    
    lut=spi.RectBivariateSpline(lon,lat,test.T)
    result = []
    for xi,yi in zip(x,y):
        result.append(lut(xi, yi)[0][0])
            
    return result


def get_nearest_continental_polygon(subduction_data,continental_polygons):

    nearest_continent_plate_id = []
    distance_to_nearest_continent = []
    
    for index,row in subduction_data.iterrows():
        
        min_distance_to_all_features = np.radians(180)
        nearest_continent = None        
        
        seed_point = pygplates.PointOnSphere(row['lat'], row['lon'])
        
        for polygon in continental_polygons:
            if polygon is not None:
                min_distance_to_feature = pygplates.GeometryOnSphere.distance(
                    polygon.get_reconstructed_geometry(),
                    seed_point,
                    min_distance_to_all_features,
                    geometry1_is_solid=True)

                # If the current geometry is nearer than all previous geometries then
                # its associated feature is the nearest feature so far.
                if min_distance_to_feature is not None:
                    min_distance_to_all_features = min_distance_to_feature
                    nearest_continent = polygon.get_feature().get_reconstruction_plate_id()
                    
        nearest_continent_plate_id.append(nearest_continent)
        distance_to_nearest_continent.append(min_distance_to_all_features*pygplates.Earth.mean_radius_in_kms)

    return nearest_continent_plate_id,distance_to_nearest_continent


### Number crunching section

In [10]:
# Create a dataframe that contains all subduction data for full sequence of time steps

# Data frame template defining the column names
DataFrameTemplate = ('lon','lat','conv_rate','conv_obliq','migr_rate',
                     'migr_obliq','arc_length','arc_azimuth',
                     'subducting_plate','overriding_plate','time')

# Create an empty dataframe to concatenate results to
df_AllTimes = pd.DataFrame(columns=DataFrameTemplate)

# list of reconstruction times
reconstruction_times = np.arange(min_time,max_time+1,time_step)

# Iterate over time steps
for reconstruction_time in reconstruction_times:
        
    # Get the subduction kinematics database 
    subduction_data = []
    output_data = sc.subduction_convergence(
                rotation_filename,
                input_topology_filename,
                threshold_sampling_distance_radians,
                reconstruction_time,
                velocity_delta_time,
                anchor_plate_id)

    # Make a flat list of subduction stats to input into the proximity test
    for data in output_data:
        subduction_data.append(data+(reconstruction_time,))
    
    # convert list array to dataframe
    df = pd.DataFrame(subduction_data, columns = DataFrameTemplate)
    
    ######################
    # Age grid sampling
    grdfile = '%s%0.0f%s' % (agegrid_filename[0],reconstruction_time,agegrid_filename[1])
    print 'working on file %s' % grdfile
    sample_result = sample_grid_using_scipy(df['lon'],df['lat'],grdfile)
    
    # Append interpolate seafloor ages to dataframe
    df['SeafloorAge'] = pd.Series(sample_result)
    
    
    ######################
    # Find nearest continental polygon
    reconstructed_continental_polygons = []
    pygplates.reconstruct(continental_polygon_features,
                          rotation_model,
                          reconstructed_continental_polygons,
                          reconstruction_time,
                          anchor_plate_id=anchor_plate_id)
    pid,dist = get_nearest_continental_polygon(df,reconstructed_continental_polygons)
    df['NearestContinentPID'] = pd.Series(pid)
    df['DistanceToContinent'] = pd.Series(dist)

    # append dataframe 
    df_AllTimes = df_AllTimes.append(df)
    

Unable to find the overriding and subducting plates of the subducting shared sub-segment "Aleutian and Bering Sea Masking Polygon" at 0.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Hikurangi& Kermadec TR" at 0.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Kalimantan subduction boundary (inferred)" at 0.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Tonga-Kermadec-MG" at 0.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Aleutian and Bering Sea Masking Polygon" at 0.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find th

working on file Data/Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_etal_2016_AgeGrid-0.nc


/usr/local/lib/python2.7/dist-packages/scipy/io/netcdf.py:317: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  ), category=RuntimeWarning)
/usr/local/lib/python2.7/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Aleutian and Bering Sea Masking Polygon" at 1.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducti

working on file Data/Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_etal_2016_AgeGrid-1.nc


Unable to find the overriding and subducting plates of the subducting shared sub-segment "Aleutian and Bering Sea Masking Polygon" at 2.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Aleutian and Bering Sea Masking Polygon" at 2.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Izu Bonin Trench" at 2.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Kalimantan subduction boundary (inferred)" at 2.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Kamchatka SZ from EarthByte COB at 0Ma -- MG 4/20/07" at 2.0Ma
    there are not exactly 2 topologies sharing the s

...

working on file Data/Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_etal_2016_AgeGrid-188.nc


Unable to find the overriding and subducting plates of the subducting shared sub-segment "East Asia subduction 250-150 Ma" at 189.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Monghol-Okosk Subduction Zone MS" at 189.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Koni-Taigonos Subduction Zone GS" at 189.0Ma
    there are not exactly 2 topologies sharing the sub-segment.


working on file Data/Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_etal_2016_AgeGrid-189.nc


Unable to find the overriding and subducting plates of the subducting shared sub-segment "Koni-Taigonos Subduction Zone GS" at 190.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Koni-Taigonos Subduction Zone GS" at 190.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "East Asia subduction 250-150 Ma" at 190.0Ma
    there are not exactly 2 topologies sharing the sub-segment.
Unable to find the overriding and subducting plates of the subducting shared sub-segment "Monghol-Okosk Subduction Zone MS" at 190.0Ma
    there are not exactly 2 topologies sharing the sub-segment.


working on file Data/Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_etal_2016_AgeGrid-190.nc
working on file Data/Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_etal_2016_AgeGrid-191.nc
working on file Data/Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_etal_2016_AgeGrid-192.nc
working on file Data/Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_etal_2016_AgeGrid-193.nc
working on file Data/Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_etal_2016_AgeGrid-194.nc
working on file Data/Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_etal_2016_AgeGrid-195.nc
working on file Data/Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_etal_2016_AgeGrid-196.nc
working on file Data/Muller_etal_2016_AREPS_Agegrids_v1.11/netCDF_0-230Ma/EarthByte_AREPS_v1.11_Muller_e

### Options for saving/loading results to/from file

In [11]:
# Save to csv file (prefered file format)
df_AllTimes.to_csv('tmp/SubductionTable_clean_%0.0f_%0.0fMa.csv' % (min_time,max_time)) 

# Save to an hdf5 file
#df_AllTimes.to_hdf('SubductionTable_%0.0f_%0.0fMa.h5' % (min_time,max_time),'SubductionTable') 

# Example of loading previously calculated results from a hdf5 file as created using the line above
#df_AllTimes = pd.read_hdf('SubductionTable_0_230Ma.h5')

#plt.scatter(df.lon,df.lat,c=df.DistanceToContinent,edgecolors='')
#plt.show()


In [12]:
# Display a preveiw of the data we just created
df_AllTimes

,DistanceToContinent,NearestContinentPID,SeafloorAge,arc_azimuth,arc_length,conv_obliq,conv_rate,lat,lon,migr_obliq,migr_rate,overriding_plate,subducting_plate,time
0,0.000000,16130.0,33.191524,7.048196,0.407674,-55.309337,6.812935,50.444919,-179.937442,164.654379,1.223722,101,901,0.0
1,104.296523,806.0,72.351119,293.688940,0.468018,-39.017566,4.843504,-37.283931,-179.904160,44.009679,-5.010784,801,901,0.0
2,1675.109671,844.0,77.421896,196.384032,0.338897,65.727968,9.177222,-11.225768,-179.705383,-135.001933,10.433777,801,901,0.0
3,147.636960,806.0,72.765439,293.547286,0.468018,-38.350411,4.914457,-36.855115,-179.669162,44.433350,-4.972855,801,901,0.0
4,170.352834,823.0,73.459981,293.408606,0.468018,-37.704138,4.985854,-36.425842,-179.436788,44.865451,-4.935034,801,901,0.0
5,1694.367534,844.0,77.323223,196.319209,0.338897,65.996958,9.157936,-11.321178,-179.373795,-134.407560,10.327195,801,901,0.0
6,0.000000,16130.0,33.191525,7.537337,0.407674,-55.516762,6.795949,50.393168,-179.302793,164.848028,1.238988,101,901,0.0
7,147.946897,823.0,74.696435,293.272837,0.468018,-37.077989,5.057658,-35.996123,-179.206970,45.306141,-4.897334,801,901,0.0
8,1714.223219,844.0,76.903078,196.253802,0.338897,66.267825,9.138799,-11.416219,-179.041987,-133.799833,10.221573,801,901,0.0
9,127.142407,823.0,75.319742,286.827791,0.490832,-30.178118,5.134455,-35.546225,-179.005424,52.117849,-4.861200,801,901,0.0
